In [ ]:
import mlflow
import pandas as pd
uri = ... # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)


In [2]:

def generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim"
):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )

        for run in runs:
            note = run.data.params.get("note")
            fusion = run.data.params.get("SAE_fusion_strategy")
            group = run.data.params.get("group_type")

            if fusion != aggregation_function or group != group_type:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dataset, dim, topk)
            model_type = "main" if note == main_note else "compare" if note == note_to_compare else None
            if not model_type:
                continue

            metrics = {
                ("G/mean", model_type): run.data.metrics.get("CommonItemsNDCG20/mean"),
                ("U/mean", model_type): run.data.metrics.get("NDCG20/mean"),
                ("U/min", model_type): run.data.metrics.get("NDCG20/min"),
                ("Pop", model_type): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Combine metrics into a dictionary
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dataset", "Dimensions", "TopK"]

    # Calculate % differences
    result_cols = []
    for metric in sorted(set(k[0] for k in df.columns)):
        main_col = (metric, "main")
        compare_col = (metric, "compare")
        percent_col = (metric, "% change")

        if main_col in df.columns and compare_col in df.columns:
            df[percent_col] = ((df[main_col] - df[compare_col]) / df[compare_col].abs()) * 100
            result_cols.extend([main_col,percent_col])
        elif main_col in df.columns:
            result_cols.append(main_col)

    # Keep only main and percent change columns, sort them by metric
    df = df[result_cols]
    df = df.round(2)
    df = df.sort_index(axis=1, level=0).sort_values(by=["Dataset", "Dimensions", "TopK"])
    

    return df.reset_index()

In [3]:
def generate_sae_table_with_comparison(experiment_ids, main_note, note_to_compare):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )

        for run in runs:
            note = run.data.params.get("note")
            if note not in [main_note, note_to_compare]:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            base_factors = float(run.data.params.get("base_factors", 0.0))
            expansion_ratio = dim / base_factors if base_factors else None
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dim, round(expansion_ratio, 3), topk)
            model_type = "main" if note == main_note else "compare"

            metrics = {
                (dataset, model_type, "CS"): run.data.metrics.get("CosineSim/test"),
                (dataset, model_type, "Deg"): run.data.metrics.get("NDCG20_Degradation/test"),
                (dataset, model_type, "Deads"): run.data.metrics.get("DeadNeurons/test")
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dimensions", "Expansion Ratio", "TopK"]

    # Calculate % differences and arrange columns
    result_cols = []
    for (dataset, metric) in sorted(set((k[0], k[2]) for k in df.columns)):
        main_col = (dataset, "main", metric)
        compare_col = (dataset, "compare", metric)
        percent_col = (dataset, "% change", metric)

        if main_col in df.columns and compare_col in df.columns:
            df[percent_col] = ((df[main_col] - df[compare_col]) / df[compare_col].abs()) * 100
            result_cols.extend([main_col, percent_col])
        elif main_col in df.columns:
            result_cols.append(main_col)

    # Finalize view: sort columns, round values
    df = df[result_cols]
    df = df.round(4)
    df = df.sort_index(axis=1, level=[0, 2]).sort_values(
        by=["Dimensions", "Expansion Ratio", "TopK"],
        ascending=[True, True, True]
    )

    return df.reset_index()

# Zmena v rekonstruction loss

Sparse Autoencodedry se nemuseji trenovat pouze na klasicke L2 loss, ale muzeme zvolit jine. Co se ukazalo, jako docela funkcni pro doporucovaci systemy, bylo pouzit cosine similarity jako loss. Podle papery to davalo opravdu dobre vysledky.



## Table of reconstrictions metrics

comparision of the L2 loss (compare) and cosine similarity (main) reconstruction loss on the ELSA dataset.

- CS - Cosine Similarity of original and reconstructed embeddings
- Deads - Percentage of dead neurons in the sparse embedding
- Deg - Degradation of NDCG between ELSA model and ELSA + Autoencoder

In [3]:

sae_experiments = ['657713966175362303', '852893065079987597']

table = generate_sae_table_with_comparison(
    sae_experiments,
    main_note="sizes_Cosine",
    note_to_compare="sizes_L2",
)
table

Dimensions Expansion Ratio TopK LastFM1k                           \
                                  % change    main % change    main   
                                        CS      CS    Deads   Deads   
0       1024             4.0   32   0.0135  0.9027 -10.4348  0.2012   
1       1024             4.0   64  -0.3697  0.9221 -20.0000  0.0664   
2       1024             4.0  128  -1.5272  0.9344 -53.8462  0.0059   
3       2048             8.0   32   0.8817  0.9104 -31.7431  0.3633   
4       2048             8.0   64   0.7415  0.9315 -22.7626  0.1938   
5       2048             8.0  128  -0.2503  0.9481 -19.1919  0.0781   
6       4096            16.0   32   1.5373  0.9116 -26.1170  0.5491   
7       4096            16.0   64   1.4121  0.9367 -32.8203  0.3623   
8       4096            16.0  128   0.2552  0.9520 -24.6575  0.2283   

                     MovieLens                                          
    % change    main  % change    main % change  main % change    main  
         Deg     Deg        CS      CS    Deads Deads      Deg     Deg  
0   -37.7869 -0.0184   -0.1005  0.9462      NaN   0.0  -0.3955 -0.0120  
1  -176.6462 -0.0182   -0.4022  0.9659      NaN   0.0   4.4958 -0.0054  
2  -162.6340 -0.0196   -1.1890  0.9796      NaN   0.0 -71.8368 -0.0039  
3  -802.4991 -0.0183   -0.2358  0.9405      NaN   0.0  -5.5714 -0.0152  
4   -11.2530 -0.0168   -0.6032  0.9600      NaN   0.0 -17.2345 -0.0082  
5    43.3454 -0.0106   -1.1276  0.9756      NaN   0.0 -91.6178 -0.0047  
6  -688.0801 -0.0180   -0.4869  0.9364   -100.0   0.0 -13.3860 -0.0174  
7 -5849.2522 -0.0133   -0.6854  0.9539      NaN   0.0 -12.9190 -0.0105  
8  -117.7475 -0.0184   -0.9140  0.9691      NaN   0.0 -51.6352 -0.0068

## Means of above table

In [4]:
table.mean()

Dimensions                          2389.333333
Expansion Ratio                        9.333333
TopK                                  74.666667
LastFM1k         % change  CS          0.299344
                 main      CS          0.927722
                 % change  Deads     -26.841489
                 main      Deads       0.227600
                 % change  Deg      -866.950400
                 main      Deg        -0.016844
MovieLens        % change  CS         -0.638289
                 main      CS          0.958578
                 % change  Deads    -100.000000
                 main      Deads       0.000000
                 % change  Deg       -28.900044
                 main      Deg        -0.009344
dtype: float64

## SAE group recommendation performance for **common features** aggregation function and **similar** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [5]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="common_features",
    main_note="sizes_Cosine_without_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim",
)
table

Dataset Dimensions TopK   G/mean            Pop         U/mean        \
                              % change  main % change  main % change  main   
0    LastFM1k       1024   32    -1.00  0.56     0.54  0.58    -0.53  0.79   
1    LastFM1k       1024   64    -3.61  0.55     1.79  0.58    -1.63  0.78   
2    LastFM1k       1024  128    -5.32  0.53     0.40  0.57    -2.27  0.77   
3    LastFM1k       2048   32    -2.25  0.56     3.42  0.63    -0.92  0.79   
4    LastFM1k       2048   64    -3.30  0.56     1.20  0.59    -1.38  0.79   
5    LastFM1k       2048  128    -3.59  0.55     2.06  0.58    -1.57  0.78   
6    LastFM1k       4096   32     1.18  0.57    -2.31  0.60     0.60  0.79   
7    LastFM1k       4096   64    -3.21  0.56     6.28  0.64    -1.22  0.79   
8    LastFM1k       4096  128    -3.43  0.56     3.64  0.60    -1.30  0.79   
9   MovieLens       1024   32    -3.29  0.55    -2.08  0.46    -2.31  0.64   
10  MovieLens       1024   64    -0.34  0.58    -4.02  0.44    -0.54  0.66   
11  MovieLens       1024  128     4.40  0.61     3.26  0.47     2.23  0.68   
12  MovieLens       2048   32   -13.56  0.51    -2.22  0.44    -8.56  0.61   
13  MovieLens       2048   64    -3.21  0.56    -0.99  0.45    -1.94  0.64   
14  MovieLens       2048  128     2.84  0.60     0.62  0.46     1.27  0.67   
15  MovieLens       4096   32    -2.98  0.57     1.18  0.47    -1.44  0.66   
16  MovieLens       4096   64    -6.91  0.56    -2.69  0.46    -4.17  0.64   
17  MovieLens       4096  128    -0.21  0.59    -2.12  0.45     0.06  0.66   

      U/min        
   % change  main  
0     -0.86  0.61  
1     -3.02  0.60  
2     -4.03  0.59  
3     -2.38  0.60  
4     -2.74  0.61  
5     -2.82  0.60  
6     -0.14  0.61  
7     -3.56  0.60  
8     -3.44  0.61  
9     -2.36  0.49  
10    -0.06  0.51  
11     3.43  0.53  
12   -12.76  0.45  
13    -2.32  0.49  
14     2.33  0.53  
15    -3.59  0.50  
16    -6.27  0.49  
17     0.54  0.51

In [6]:
table.groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/2181786212.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -2.725556  0.555556  1.891111  0.596667   
MovieLens  2389.333333  74.666667 -2.584444  0.570000 -1.006667  0.455556   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -1.135556  0.785556 -2.554444  0.603333  
MovieLens -1.711111  0.651111 -2.340000  0.500000

In [7]:
table.groupby(["Dataset"]).std()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/2762490340.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby(["Dataset"]).std()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   1354.624671  42.332021  1.863418  0.011304  2.420565  0.023979   
MovieLens  1354.624671  42.332021  5.351418  0.030000  2.275132  0.011304   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k   0.809276  0.007265  1.276001  0.007071  
MovieLens  3.208693  0.020883  4.926703  0.024495

## SAE group recommendation performance for **common features** aggregation function and **random** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [8]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="common_features",
    main_note="sizes_Cosine_without_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="random",
)

In [9]:
table.groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/2181786212.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -4.448889  0.460000  1.713333  0.656667   
MovieLens  2389.333333  74.666667 -5.532222  0.502222 -1.983333  0.466667   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -1.786667  0.724444 -3.830000  0.505556  
MovieLens -3.498889  0.596667 -5.232222  0.436667

## SAE group recommendation performance for **common features** aggregation function and **divergen** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [10]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="common_features",
    main_note="sizes_Cosine_without_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="outlier",
)
table

Dataset Dimensions TopK   G/mean            Pop         U/mean        \
                              % change  main % change  main % change  main   
0    LastFM1k       1024   32    -3.45  0.38     0.47  0.60    -1.47  0.68   
1    LastFM1k       1024   64    -4.78  0.35     1.01  0.58    -2.70  0.66   
2    LastFM1k       1024  128    -6.06  0.34     0.63  0.57    -3.06  0.65   
3    LastFM1k       2048   32    -2.34  0.42     5.55  0.68    -0.33  0.71   
4    LastFM1k       2048   64    -7.42  0.37     1.79  0.62    -3.01  0.68   
5    LastFM1k       2048  128    -3.67  0.36     3.25  0.61    -1.69  0.67   
6    LastFM1k       4096   32    -2.85  0.42    -0.42  0.64     0.09  0.71   
7    LastFM1k       4096   64     1.21  0.43     9.17  0.68     0.41  0.71   
8    LastFM1k       4096  128    -2.44  0.39     5.74  0.63    -0.69  0.69   
9   MovieLens       1024   32     6.03  0.29    11.33  0.35     2.35  0.46   
10  MovieLens       1024   64    -6.34  0.32    -4.97  0.34    -2.25  0.50   
11  MovieLens       1024  128    15.00  0.43    12.23  0.41     6.74  0.59   
12  MovieLens       2048   32   -32.61  0.22    -2.84  0.31   -21.62  0.40   
13  MovieLens       2048   64   -17.82  0.25    -8.10  0.31    -8.12  0.45   
14  MovieLens       2048  128    -2.15  0.39    -2.45  0.38    -0.12  0.55   
15  MovieLens       4096   32    -1.26  0.42     9.26  0.42    -1.32  0.57   
16  MovieLens       4096   64    -2.70  0.34     6.61  0.37    -4.06  0.49   
17  MovieLens       4096  128     1.09  0.38    -2.98  0.38     0.74  0.53   

      U/min        
   % change  main  
0     -2.38  0.43  
1     -3.65  0.41  
2     -4.80  0.40  
3     -3.08  0.45  
4     -8.39  0.42  
5     -3.39  0.41  
6     -4.10  0.45  
7     -1.16  0.46  
8     -3.24  0.43  
9      5.86  0.28  
10    -3.52  0.32  
11    12.93  0.41  
12   -33.59  0.22  
13   -14.52  0.26  
14    -1.91  0.37  
15    -4.01  0.39  
16    -6.06  0.31  
17     0.64  0.35

In [11]:
table.groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/2181786212.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -3.533333  0.384444  3.021111  0.623333   
MovieLens  2389.333333  74.666667 -4.528889  0.337778  2.010000  0.363333   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -1.383333  0.684444 -3.798889  0.428889  
MovieLens -3.073333  0.504444 -4.908889  0.323333

## SAE group recommendation performance for **average** aggregation function and **similar** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [12]:
experiments = ['523100174176986081', '333391697323445885']
generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_Cosine_with_acts_normalized",
    note_to_compare="sizes_L2_with_acts",
    group_type="sim",
).groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/2400983973.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667  0.124444  0.590000  0.688889  0.573333   
MovieLens  2389.333333  74.666667 -0.851111  0.641111 -1.503333  0.495556   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k   0.061111  0.805556 -0.143333  0.633333  
MovieLens -0.392222  0.698889 -0.808889  0.553333

## SAE group recommendation performance for **average** aggregation function and **random** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [13]:
experiments = ['523100174176986081', '333391697323445885']
generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_Cosine_with_acts",
    note_to_compare="sizes_L2_with_acts",
    group_type="random",
).groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/3770659005.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -0.455556  0.497778 -2.570000  0.620000   
MovieLens  2389.333333  74.666667 -2.230000  0.616667 -3.285556  0.527778   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -0.532222  0.747778  0.116667  0.545556  
MovieLens -1.273333  0.681111 -1.542222  0.531111

## SAE group recommendation performance for **average** aggregation function and **random** groups

Comparing the L2 variant (compare) with the cosine similarity variant (main) 

In [14]:
experiments = ['523100174176986081', '333391697323445885']
generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_Cosine_with_acts_normalized",
    note_to_compare="sizes_L2_with_acts",
    group_type="outlier",
).groupby(["Dataset"]).mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_38945/982946448.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby(["Dataset"]).mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667  0.216667  0.413333  0.883333  0.587778   
MovieLens  2389.333333  74.666667 -4.058889  0.524444 -3.165556  0.462222   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k   0.007778  0.716667 -0.255556  0.445556  
MovieLens -1.865556  0.654444 -3.293333  0.470000